In [1]:
!pip install tensorflow==2.0.0-alpha0

     |████████████████████████████████| 79.9MB 1.3MB/s 
     |████████████████████████████████| 419kB 25.3MB/s 
     |████████████████████████████████| 3.0MB 31.3MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2019-07-27 15:52:37--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2019-07-27 15:52:37 (107 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [5]:
model = Sequential()
model.add(Embedding(total_words,100,input_length = max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(total_words/2,activation='relu',kernel_regularizer = regularizers.l2(0.01)))
model.add(Dense(total_words,activation='softmax'))
# Pick an optimizer
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, 10, 150)           150600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 150)           0         
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (None, 64)                55040     
_________________________________________________________________
dense_2 (Dense)              (None, 1605)              104325    
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              5156866   
Total params: 5,787,931
Trainable params: 5,787,931
Non-trainable params: 0
____________________________________________

In [6]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
15462/15462 [==============================] - 54s 3ms/sample - loss: 6.8955 - accuracy: 0.0224
Epoch 2/100
15462/15462 [==============================] - 54s 3ms/sample - loss: 6.5103 - accuracy: 0.0209
Epoch 3/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 6.4398 - accuracy: 0.0222
Epoch 4/100
15462/15462 [==============================] - 53s 3ms/sample - loss: 6.3711 - accuracy: 0.0259
Epoch 5/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 6.2777 - accuracy: 0.0305
Epoch 6/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 6.1882 - accuracy: 0.0314
Epoch 7/100
15462/15462 [==============================] - 51s 3ms/sample - loss: 6.1207 - accuracy: 0.0374
Epoch 8/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 6.0513 - accuracy: 0.0392
Epoch 9/100
15462/15462 [==============================] - 53s 3ms/sample - loss: 5.9928 - accuracy: 0.0399
Epoch 10/100
15462/15462 [==

In [1]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

NameError: ignored

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)